In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [7]:
for soft in ['FSL']:
    for f in [5]:
        for p in [0,6,24]:
            for h in [0,1]:
                df = pd.read_csv(f'../figures/validation_{soft}_{f}_{p}_{h}_roi_unilat.csv')
                print(df.groupby(['contrast']).mean())
                
                #fig = plt.figure(figsize=(16,10))

                #sns.set_style("whitegrid")
                #sns.boxplot(data=df, x="contrast", y="percent_overlap")
                #plt.show()

          Unnamed: 0  percent_overlap
contrast                             
cue           2997.0         0.904759
lf            2998.0         0.549964
lh            2999.0         0.826139
rf            3000.0         0.520952
rh            3001.0         0.805597
t             3002.0         0.819304
          Unnamed: 0  percent_overlap
contrast                             
cue           2997.0         0.954461
lf            2998.0         0.487836
lh            2999.0         0.740444
rf            3000.0         0.438247
rh            3001.0         0.689215
t             3002.0         0.741608
          Unnamed: 0  percent_overlap
contrast                             
cue           2997.0         0.866503
lf            2998.0         0.450081
lh            2999.0         0.815933
rf            3000.0         0.356787
rh            3001.0         0.812827
t             3002.0         0.785698
          Unnamed: 0  percent_overlap
contrast                             
cue         

/var/folders/k4/cck9mrt578q044z6x6zmsnc400m_jt/T/ipykernel_62436/3022286824.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(df.groupby(['contrast']).mean())
/var/folders/k4/cck9mrt578q044z6x6zmsnc400m_jt/T/ipykernel_62436/3022286824.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(df.groupby(['contrast']).mean())
/var/folders/k4/cck9mrt578q044z6x6zmsnc400m_jt/T/ipykernel_62436/3022286824.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns w

In [8]:
from nilearn import datasets
d = datasets.fetch_atlas_juelich('prob-2mm')

/Users/egermani/miniforge3/envs/workEnv/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


{'filename': <nibabel.nifti1.Nifti1Image at 0x155b76e80>,
 'maps': <nibabel.nifti1.Nifti1Image at 0x1578e06d0>,
 'labels': ['Background',
  'GM Amygdala_centromedial group',
  'GM Amygdala_laterobasal group',
  'GM Amygdala_superficial group',
  'GM Anterior intra-parietal sulcus hIP1',
  'GM Anterior intra-parietal sulcus hIP2',
  'GM Anterior intra-parietal sulcus hIP3',
  "GM Broca's area BA44",
  "GM Broca's area BA45",
  'GM Hippocampus cornu ammonis',
  'GM Hippocampus dentate gyrus',
  'GM Hippocampus entorhinal cortex',
  'GM Hippocampus hippocampal-amygdaloid transition area',
  'GM Hippocampus subiculum',
  'GM Inferior parietal lobule PF',
  'GM Inferior parietal lobule PFcm',
  'GM Inferior parietal lobule PFm',
  'GM Inferior parietal lobule PFop',
  'GM Inferior parietal lobule PFt',
  'GM Inferior parietal lobule PGp',
  'GM Inferior parietal lobule Pga',
  'GM Insula Id1',
  'GM Insula Ig1',
  'GM Insula Ig2',
  'GM Lateral geniculate body',
  'GM Mamillary body',
  'GM